A very basic approach to classify a sentence as positive or negative. 

In [1]:
def find_sentiment(sentence, pos, neg):
    
    #Splitting the sentence by space and converting it to a set to remove duplicate words
    sentence = sentence.split()
    sentence = set(sentence)
    
    num_common_pos = len(sentence.intersection(pos))
    num_common_neg = len(sentence.intersection(neg))

    print(f"Input sentence : {sentence}")
    print(f"Number of positive words : {num_common_pos}")
    print(f"Number of negative words : {num_common_neg}")
    
    if num_common_pos > num_common_neg:
        print(f"Sentiment of the sentence : Positive")
        return "positive"
    if num_common_pos < num_common_neg:
        print(f"Sentiment of the sentence : Negative")
        return "negative"
    print(f"Sentiment of the sentence : Neutral")
    return "neutral"

Improving the text splitting method using nltk

In [2]:
from nltk.tokenize import word_tokenize

In [3]:
sentence = "hi, how are you?"

In [8]:
#Using sentence.split
print("Using sentence.split()")
print(sentence.split())
print()
#Using nltk word tokenizer
print("Using nltk word tokenizer")
print(word_tokenize(sentence))

Using sentence.split()
['hi,', 'how', 'are', 'you?']

Using nltk word tokenizer
['hi', ',', 'how', 'are', 'you', '?']


BAG OF WORDS - A huge sparse matrix that stores the count of all the individual words present in the corpus

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
corpus = [
    "hello, how are you?",
    "im getting bored at home. And you? What do you think?",
    "did you know about counts",
    "let's see if this works!",
    "YES!!!!"
]

In [24]:
ctv = CountVectorizer()

ctv.fit(corpus)

corpus_transformed = ctv.transform(corpus)

Vocabulary

In [29]:
ctv.vocabulary_

{'hello': 9,
 'how': 11,
 'are': 2,
 'you': 22,
 'im': 13,
 'getting': 8,
 'bored': 4,
 'at': 3,
 'home': 10,
 'and': 1,
 'what': 19,
 'do': 7,
 'think': 17,
 'did': 6,
 'know': 14,
 'about': 0,
 'counts': 5,
 'let': 15,
 'see': 16,
 'if': 12,
 'this': 18,
 'works': 20,
 'yes': 21}

In [30]:
print(corpus_transformed)

  (0, 2)	1
  (0, 9)	1
  (0, 11)	1
  (0, 22)	1
  (1, 1)	1
  (1, 3)	1
  (1, 4)	1
  (1, 7)	1
  (1, 8)	1
  (1, 10)	1
  (1, 13)	1
  (1, 17)	1
  (1, 19)	1
  (1, 22)	2
  (2, 0)	1
  (2, 5)	1
  (2, 6)	1
  (2, 14)	1
  (2, 22)	1
  (3, 12)	1
  (3, 15)	1
  (3, 16)	1
  (3, 18)	1
  (3, 20)	1
  (4, 21)	1


In the above response (x,y) x indicates the sample index and y indicates the id associated to the word in the vocabulary. The number of times that word has occured in that sentence is also printed above.

Using Count Vectorizer with nltk tokenizer

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize

In [32]:
corpus = [
"hello, how are you?",
"im getting bored at home. And you? What do you think?",
"did you know about counts",
"let's see if this works!",
"YES!!!!"
]

In [33]:
ctv = CountVectorizer(tokenizer=word_tokenize, token_pattern=None)

ctv.fit(corpus)

corpus_transformed = ctv.transform(corpus)

print(ctv.vocabulary_)

{'hello': 14, ',': 2, 'how': 16, 'are': 7, 'you': 27, '?': 4, 'im': 18, 'getting': 13, 'bored': 9, 'at': 8, 'home': 15, '.': 3, 'and': 6, 'what': 24, 'do': 12, 'think': 22, 'did': 11, 'know': 19, 'about': 5, 'counts': 10, 'let': 20, "'s": 1, 'see': 21, 'if': 17, 'this': 23, 'works': 25, '!': 0, 'yes': 26}


In the above sample the punctuations are also included as tokens.

MODELS

In [37]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn import linear_model, metrics, model_selection
from sklearn.feature_extraction.text import CountVectorizer


In [38]:
df = pd.read_csv(r'imdb.csv')
print(f"Shape of data : {df.shape}")

df.sentiment = df.sentiment.apply(
    lambda x:1 if x=="positive" else 0
)

df['kfold'] = -1

Shape of data : (50000, 2)


In [39]:
df = df.sample(frac=1).reset_index(drop=True)

In [45]:
y = df.sentiment.values

kf = model_selection.StratifiedKFold(n_splits=5)

for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'kfold'] = f

In [50]:
for fold_ in range(5):
    train_df = df[df.kfold != fold_].reset_index(drop=True)
    test_df = df[df.kfold == fold_].reset_index(drop=True)

    count_vec = CountVectorizer(
    tokenizer=word_tokenize,
    token_pattern=None
    )

    count_vec.fit(train_df.review)

    xtrain = count_vec.transform(train_df.review)
    xtest = count_vec.transform(test_df.review)

    model = linear_model.LogisticRegression()

    model.fit(xtrain, train_df.sentiment)

    preds = model.predict(xtest)

    accuracy = metrics.accuracy_score(test_df.sentiment, preds)

    print(f"Fold : {fold_}")
    print(f"Accuracy = {accuracy}")
    print("")

d:\Anaconda\envs\nlp\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold : 0
Accuracy = 0.8906



d:\Anaconda\envs\nlp\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold : 1
Accuracy = 0.8986



d:\Anaconda\envs\nlp\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold : 2
Accuracy = 0.894



d:\Anaconda\envs\nlp\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold : 3
Accuracy = 0.8889

Fold : 4
Accuracy = 0.8886



d:\Anaconda\envs\nlp\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [51]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn import linear_model, metrics, model_selection, naive_bayes
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv(r'imdb.csv')
print(f"Shape of data : {df.shape}")

df.sentiment = df.sentiment.apply(
    lambda x:1 if x=="positive" else 0
)

df['kfold'] = -1

df = df.sample(frac=1).reset_index(drop=True)

y = df.sentiment.values

kf = model_selection.StratifiedKFold(n_splits=5)

for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'kfold'] = f


for fold_ in range(5):
    train_df = df[df.kfold != fold_].reset_index(drop=True)
    test_df = df[df.kfold == fold_].reset_index(drop=True)

    count_vec = CountVectorizer(
    tokenizer=word_tokenize,
    token_pattern=None
    )

    count_vec.fit(train_df.review)

    xtrain = count_vec.transform(train_df.review)
    xtest = count_vec.transform(test_df.review)

    #NAIVE BAYES MODEL
    model = naive_bayes.MultinomialNB()

    model.fit(xtrain, train_df.sentiment)

    preds = model.predict(xtest)

    accuracy = metrics.accuracy_score(test_df.sentiment, preds)

    print(f"Fold : {fold_}")
    print(f"Accuracy = {accuracy}")
    print("")


Shape of data : (50000, 2)
Fold : 0
Accuracy = 0.8421

Fold : 1
Accuracy = 0.8439

Fold : 2
Accuracy = 0.839

Fold : 3
Accuracy = 0.8406

Fold : 4
Accuracy = 0.8528



TFIDF

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize

In [57]:
corpus = [
    "hello, how are you?",
    "im getting bored at home. And you? What do you think?",
    "did you know about counts",
    "let's see if this works!",
    "YES!!!!"
]

tfv = TfidfVectorizer(tokenizer=word_tokenize, token_pattern=None)

tfv.fit(corpus)

corpus_transformed = tfv.transform(corpus)

print("VOCAB")
print(tfv.vocabulary_)
print()
print(f"TFIDF CORPUS TRANSFORMED")
print(corpus_transformed)

VOCAB
{'hello': 14, ',': 2, 'how': 16, 'are': 7, 'you': 27, '?': 4, 'im': 18, 'getting': 13, 'bored': 9, 'at': 8, 'home': 15, '.': 3, 'and': 6, 'what': 24, 'do': 12, 'think': 22, 'did': 11, 'know': 19, 'about': 5, 'counts': 10, 'let': 20, "'s": 1, 'see': 21, 'if': 17, 'this': 23, 'works': 25, '!': 0, 'yes': 26}

TFIDF CORPUS TRANSFORMED
  (0, 27)	0.2965698850220162
  (0, 16)	0.4428321995085722
  (0, 14)	0.4428321995085722
  (0, 7)	0.4428321995085722
  (0, 4)	0.35727423026525224
  (0, 2)	0.4428321995085722
  (1, 27)	0.35299699146792735
  (1, 24)	0.2635440111190765
  (1, 22)	0.2635440111190765
  (1, 18)	0.2635440111190765
  (1, 15)	0.2635440111190765
  (1, 13)	0.2635440111190765
  (1, 12)	0.2635440111190765
  (1, 9)	0.2635440111190765
  (1, 8)	0.2635440111190765
  (1, 6)	0.2635440111190765
  (1, 4)	0.42525129752567803
  (1, 3)	0.2635440111190765
  (2, 27)	0.31752680284846835
  (2, 19)	0.4741246485558491
  (2, 11)	0.4741246485558491
  (2, 10)	0.4741246485558491
  (2, 5)	0.4741246485558491

In the above response (x,y) x indicates the sample index and y indicates the id associated to the word in the vocabulary. The tfidf score of that word is mentioned after that.

{'hello': 14,
 ',': 2,
 'how': 16,
 'are': 7,
 'you': 27,
 '?': 4,
 'im': 18,
 'getting': 13,
 'bored': 9,
 'at': 8,
 'home': 15,
 '.': 3,
 'and': 6,
 'what': 24,
 'do': 12,
 'think': 22,
 'did': 11,
 'know': 19,
 'about': 5,
 'counts': 10,
 'let': 20,
 "'s": 1,
 'see': 21,
 'if': 17,
 'this': 23,
 'works': 25,
 '!': 0,
 'yes': 26}

Logistic Regression using TFIDF vectorizer

In [58]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn import linear_model, metrics, model_selection
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv(r'imdb.csv')
print(f"Shape of data : {df.shape}")

df.sentiment = df.sentiment.apply(
    lambda x:1 if x=="positive" else 0
)

df['kfold'] = -1

df = df.sample(frac=1).reset_index(drop=True)

y = df.sentiment.values

kf = model_selection.StratifiedKFold(n_splits=5)

for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'kfold'] = f


for fold_ in range(5):
    train_df = df[df.kfold != fold_].reset_index(drop=True)
    test_df = df[df.kfold == fold_].reset_index(drop=True)

    tfidf_vec = TfidfVectorizer(
    tokenizer=word_tokenize,
    token_pattern=None
    )

    tfidf_vec.fit(train_df.review)

    xtrain = tfidf_vec.transform(train_df.review)
    xtest = tfidf_vec.transform(test_df.review)

    #LOGISTIC REGRESSION MODEL
    model = linear_model.LogisticRegression()

    model.fit(xtrain, train_df.sentiment)

    preds = model.predict(xtest)

    accuracy = metrics.accuracy_score(test_df.sentiment, preds)

    print(f"Fold : {fold_}")
    print(f"Accuracy = {accuracy}")
    print("")


Shape of data : (50000, 2)


d:\Anaconda\envs\nlp\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold : 0
Accuracy = 0.8961



d:\Anaconda\envs\nlp\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold : 1
Accuracy = 0.8942



d:\Anaconda\envs\nlp\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold : 2
Accuracy = 0.8964



d:\Anaconda\envs\nlp\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold : 3
Accuracy = 0.8992

Fold : 4
Accuracy = 0.8995



N-Grams <br>

ngrams are combination of n words in order. 'n' can be 1 , 2 ,3 etc.. <br>
<br>
n = 1 -> unigram <br>
n = 2 -> bigram <br>
n = 3 -> trigram <br>
<br> <br>

Eg. Hello, how are you. <br>

unigram - 'hello' ,'how', 'are', 'you' <br>
bigram - 'hello how', 'how are', 'are you' <br>
trigram - 'hello how are', 'how are you' <br>



In [62]:
from nltk import ngrams
from nltk.tokenize import word_tokenize

sentence = "hi, how are you?"

N = 3

tokenized_sentence = word_tokenize(sentence)

for n in range(1,N+1):
    print(f"N = {n}")
    ngrams_ = list(ngrams(tokenized_sentence, n))

    print(ngrams_)

N = 1
[('hi',), (',',), ('how',), ('are',), ('you',), ('?',)]
N = 2
[('hi', ','), (',', 'how'), ('how', 'are'), ('are', 'you'), ('you', '?')]
N = 3
[('hi', ',', 'how'), (',', 'how', 'are'), ('how', 'are', 'you'), ('are', 'you', '?')]


Logistic regression with TFIDF and ngrams

Trigrams

In [64]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn import linear_model, metrics, model_selection
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv(r'imdb.csv')
print(f"Shape of data : {df.shape}")

df.sentiment = df.sentiment.apply(
    lambda x:1 if x=="positive" else 0
)

df['kfold'] = -1

df = df.sample(frac=1).reset_index(drop=True)

y = df.sentiment.values

kf = model_selection.StratifiedKFold(n_splits=5)

for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'kfold'] = f


for fold_ in range(5):
    train_df = df[df.kfold != fold_].reset_index(drop=True)
    test_df = df[df.kfold == fold_].reset_index(drop=True)

    #Using Ngrams during vectorization
    tfidf_vec = TfidfVectorizer(
    tokenizer=word_tokenize,
    token_pattern=None,
    ngram_range=(1,3)
    )

    tfidf_vec.fit(train_df.review)

    xtrain = tfidf_vec.transform(train_df.review)
    xtest = tfidf_vec.transform(test_df.review)

    #LOGISTIC REGRESSION MODEL
    model = linear_model.LogisticRegression()

    model.fit(xtrain, train_df.sentiment)

    preds = model.predict(xtest)

    accuracy = metrics.accuracy_score(test_df.sentiment, preds)

    print(f"Fold : {fold_}")
    print(f"Accuracy = {accuracy}")
    print("")


Shape of data : (50000, 2)
Fold : 0
Accuracy = 0.8937



d:\Anaconda\envs\nlp\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold : 1
Accuracy = 0.8926

Fold : 2
Accuracy = 0.8975



d:\Anaconda\envs\nlp\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold : 3
Accuracy = 0.8883



d:\Anaconda\envs\nlp\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold : 4
Accuracy = 0.8896



Bigrams

In [65]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn import linear_model, metrics, model_selection
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv(r'imdb.csv')
print(f"Shape of data : {df.shape}")

df.sentiment = df.sentiment.apply(
    lambda x:1 if x=="positive" else 0
)

df['kfold'] = -1

df = df.sample(frac=1).reset_index(drop=True)

y = df.sentiment.values

kf = model_selection.StratifiedKFold(n_splits=5)

for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'kfold'] = f


for fold_ in range(5):
    train_df = df[df.kfold != fold_].reset_index(drop=True)
    test_df = df[df.kfold == fold_].reset_index(drop=True)

    #Using Ngrams during vectorization
    tfidf_vec = TfidfVectorizer(
    tokenizer=word_tokenize,
    token_pattern=None,
    ngram_range=(1,2)
    )

    tfidf_vec.fit(train_df.review)

    xtrain = tfidf_vec.transform(train_df.review)
    xtest = tfidf_vec.transform(test_df.review)

    #LOGISTIC REGRESSION MODEL
    model = linear_model.LogisticRegression()

    model.fit(xtrain, train_df.sentiment)

    preds = model.predict(xtest)

    accuracy = metrics.accuracy_score(test_df.sentiment, preds)

    print(f"Fold : {fold_}")
    print(f"Accuracy = {accuracy}")
    print("")


Shape of data : (50000, 2)


d:\Anaconda\envs\nlp\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold : 0
Accuracy = 0.8959

Fold : 1
Accuracy = 0.8994



d:\Anaconda\envs\nlp\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold : 2
Accuracy = 0.9046

Fold : 3
Accuracy = 0.8969

Fold : 4
Accuracy = 0.9014



d:\Anaconda\envs\nlp\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Stemming and Lemmatization <br>

It's the process of reducing a word to its smallest form. In stemming, the proccessed word in called <b>stemmed word</b> and in lemmatization the proccessed word is called <b>lemma</b>. <br><br>

For stemming we will use <b>smowball stemmer</b> and for lemmatization we will use <b>wordnet lemmatizer</b>

In [66]:
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

In [68]:
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

words = ["fishing","fishes","fished"]

for word in words:
    print(f"word = {word}")
    print(f"Stemmed word = {stemmer.stem(word)}")
    print(f"Lemma = {lemmatizer.lemmatize(word)}")
    print("")

word = fishing
Stemmed word = fish
Lemma = fishing

word = fishes
Stemmed word = fish
Lemma = fish

word = fished
Stemmed word = fish
Lemma = fished



<b>Non Negative Matrix Factorization (NMF)</b> or <b>Latent Semantic Analysis (LSA)</b> can be used for <b>Topic Extraction</b> <br><br>
They are decomposition techniques which reduce the data to a given number of components. Any sparse matrix obtained from CountVectorizer or TfidfVectorizer can be fit on this,

In [77]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn import decomposition
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = pd.read_csv(r"imdb.csv", nrows=10000)
corpus = corpus.review.values

#Init tokenizer
tfv = TfidfVectorizer(tokenizer=word_tokenize,token_pattern=None)

#Fit on corpus
tfv.fit(corpus)

#Transform the corpus
corpus_transformed = tfv.transform(corpus)

#Using SVD with 10 components
svd = decomposition.TruncatedSVD(n_components=10)

#Fit transform
corpus_svd = svd.fit(corpus_transformed)

sample_index = 0
feature_scores = dict(
    zip(
        tfv.get_feature_names_out(),
        corpus_svd.components_[sample_index]
    )
)

N = 5
print(sorted(feature_scores, key=feature_scores.get, reverse=True)[:5])

['the', ',', '.', 'a', 'and']


In [96]:
N = 5
for sample_index in range(5):

    feature_scores = dict(
        zip(
        tfv.get_feature_names_out(),
        corpus_svd.components_[sample_index]
        )
     )
    
    print(sorted(feature_scores,key=feature_scores.get,reverse=True)[:N])


['the', ',', '.', 'a', 'and']
['br', '<', '>', '/', '-']
['i', 'movie', '!', 'it', 'was']
[',', '!', "''", '``', 'you']
['!', 'the', "''", '``', '...']


In [97]:
import re, string

def clean_text(s):
    s = s.split()
    
    s = " ".join(s)
    s = re.sub(f'[{re.escape(string.punctuation)}]', '', s)

    return s

In [102]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn import decomposition
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = pd.read_csv(r"imdb.csv", nrows=10000)
corpus.loc[:, "review"] = corpus.review.apply(clean_text)
corpus = corpus.review.values

#Init tokenizer
tfv = TfidfVectorizer(tokenizer=word_tokenize,token_pattern=None)

#Fit on corpus
tfv.fit(corpus)

#Transform the corpus
corpus_transformed = tfv.transform(corpus)

#Using SVD with 10 components
svd = decomposition.TruncatedSVD(n_components=10)

#Fit transform
corpus_svd = svd.fit(corpus_transformed)

sample_index = 0
feature_scores = dict(
    zip(
        tfv.get_feature_names_out(),
        corpus_svd.components_[sample_index]
    )
)

N = 5
print(sorted(feature_scores, key=feature_scores.get, reverse=True)[:5])

['the', 'a', 'and', 'of', 'to']


In [103]:
N = 5
for sample_index in range(5):

    feature_scores = dict(
        zip(
        tfv.get_feature_names_out(),
        corpus_svd.components_[sample_index]
        )
     )
    
    print(sorted(feature_scores,key=feature_scores.get,reverse=True)[:N])


['the', 'a', 'and', 'of', 'to']
['i', 'movie', 'it', 'was', 'this']
['the', 'was', 'i', 'were', 'of']
['her', 'was', 'she', 'i', 'he']
['br', 'to', 'they', 'he', 'show']


In [109]:
a